## Exporting models for use on client-side apps

## Preprocessing models

My model was using transfer learning from Resnet-50. So, I will need to either
- create a model with both, or
- extract both models separately and use the predictions of one in the other

Architecturally, it makes more sense to go the later way, because then the last layer can be easily replaced while using this for other apps

#### Loading the saved weights for the models

In [1]:
from keras.models import load_model

base_resnet = load_model('./keras_models/base_resnet.hdf5')
last_layers_model = load_model('./keras_models/weights.extra_layers.hdf5')
last_layers_model.summary()

## ignore the warning for base_resnet!

Using TensorFlow backend.
/home/akshata/.conda/envs/dog-project/lib/python3.6/site-packages/keras/models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 500)               1024500   
_________________________________________________________________
dropout_7 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 133)               66633     
Total params: 1,091,133.0
Trainable params: 1,091,133.0
Non-trainable params: 0.0
_________________________________________________________________


#### Testing the workflow

In [2]:
import numpy as np
dog_names = np.load('./labels/dog_names.npy')
print('Number of dogs:', len(dog_names))

Number of dogs: 133


In [3]:
### function converts an image path into a tensor that can be input into the resnet model
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [5]:
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image    
import numpy as np

def predict_breed(img_path):
    tensor = path_to_tensor(img_path)
    bottleneck_feature = base_resnet.predict(preprocess_input(tensor))
    predicted_vector = last_layers_model.predict(bottleneck_feature)
    return dog_names[np.argmax(predicted_vector)]

In [6]:
predict_breed('./dogImages/001.Affenpinscher/Affenpinscher_00038.jpg')

'Affenpinsche'

## Android

Deploying to android requires a .pb format. Use the following function, to convert your model 

In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [7]:
import os
from tensorflow.python.framework import graph_util, graph_io
from tensorflow.python.tools import import_pb_to_tensorboard

def keras_to_tensorflow(keras_model, 
        output_dir, 
        model_name,
        out_prefix="output_", 
        log_tensorboard=True):

    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], 
            out_prefix + str(i + 1))

    sess = K.get_session()


    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(
            sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, 
        name=model_name, as_text=False)

    if log_tensorboard:
        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name),
            output_dir)

In [12]:
## save both models
import os
import tensorflow as tf
from keras import backend as K

keras_to_tensorflow(base_resnet, 'android', 'base_resnet_model.pb')
keras_to_tensorflow(last_layers_model, 'android', 'last_layers_model.pb')

INFO:tensorflow:Froze 318 variables.
INFO:tensorflow:Converted 318 variables to const ops.
Model Imported. Visualize by running: tensorboard --logdir=android
INFO:tensorflow:Froze 318 variables.
INFO:tensorflow:Converted 318 variables to const ops.
Model Imported. Visualize by running: tensorboard --logdir=android


## IOS Core ML (work-in-progress)

In [10]:
import coremltools

def saveCoreMLModel(model):
    coreml_model = coremltools.converters.keras.convert(model,
    input_names=['input'], output_names=['probs'],
        image_input_names='input',
        predicted_feature_name='dogBreed',
        class_labels='') ## fix
    coreml_model.save('resnet50custom.mlmodel')
    print('coreml saved')

ModuleNotFoundError: No module named 'coremltools'